In [1]:
import os

os.chdir('../..')

In [2]:
from os.path import join

import numpy as np
import pandas as pd

import modules.point_processing as pp

In [3]:
trial_name = '2014-12-03_P007_Post_000'

In [4]:
df_hypo = pd.read_pickle('data/kinect/processed/hypothesis/' + trial_name + '.pkl')
df_selected = pd.read_pickle('data/kinect/best_pos/' + trial_name + '.pkl')

df_truth = pd.read_pickle('results/dataframes/df_truth.pkl').loc[trial_name]

In [5]:
foot_parts = ['L_FOOT', 'R_FOOT']

In [6]:
# Drop frames where head or feet are null
df_truth = df_truth.dropna(subset=['HEAD'] + foot_parts)

In [7]:
# Frames with ground truth for head and foot
frames = df_truth.index.values

In [8]:
df_hypo = df_hypo.loc[frames]
df_selected = df_selected.loc[frames]

In [9]:
df_truth_modified = pd.DataFrame(index=frames, columns=foot_parts)

In [10]:
for frame in frames:
    
    foot_proposals = df_hypo.loc[frame, 'FOOT']
    true_foot_l, true_foot_r = df_truth.loc[frame, foot_parts]
        
    proposal_closest_l, _ = pp.closest_point(foot_proposals, true_foot_l.reshape(1, -1))
    proposal_closest_r, _ = pp.closest_point(foot_proposals, true_foot_r.reshape(1, -1))
    
    df_truth_modified.loc[frame, foot_parts[0]] = proposal_closest_l
    df_truth_modified.loc[frame, foot_parts[1]] = proposal_closest_r

In [11]:
pp.position_accuracy(np.stack(df_truth_modified.L_FOOT), np.stack(df_truth.L_FOOT))

0.808955223880597

In [12]:
pp.position_accuracy(np.stack(df_truth_modified.R_FOOT), np.stack(df_truth.R_FOOT))

0.7761194029850746

In [13]:
df_selected_assigned = pd.DataFrame(index=frames, columns=foot_parts)

for frame in frames:
    points_selected = np.stack(df_selected.loc[frame][foot_parts])
    points_truth =  np.stack(df_truth_modified.loc[frame][foot_parts])
    
    assigned_l, assigned_r = pp.correspond_points(points_truth, points_selected)
    
    df_selected_assigned.loc[frame, foot_parts[0]] = assigned_l
    df_selected_assigned.loc[frame, foot_parts[1]] = assigned_r

In [14]:
pp.position_accuracy(np.stack(df_selected_assigned.L_FOOT), np.stack(df_truth_modified.L_FOOT))

0.9880597014925373

In [15]:
pp.position_accuracy(np.stack(df_selected_assigned.R_FOOT), np.stack(df_truth_modified.R_FOOT))

0.982089552238806